In [1]:
%load_ext autoreload
%autoreload 2
import sys

import folium
import geopandas as gpd
import pandas as pd

sys.path.append("/home/ppeter/repo/mcr-py/src")

from package.geometa import GeoMeta
from package.logger import setup

setup("INFO")

In [2]:
# All variables arguments used in this notebook
city_name = "Berlin"
geometa_path = f"../data/stateful_variables/{city_name.lower()}_geometa.pkl"
landuse_file = f"../data/city_data/land_use_{city_name.lower()}.geojson"

In [3]:
geo_meta = GeoMeta.load(geometa_path)

In [4]:
land_use = gpd.read_file(
    "/data/passive/research-data/general-data/u2018_clc2018_v2020_20u1/DATA/U2018_CLC2018_V2020_20u1.gpkg"
)

In [5]:
land_use.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

In [6]:
polygon = geo_meta.unbuffered_boundary
boundary_gdf = gpd.GeoDataFrame(geometry=[polygon])
boundary_gdf = boundary_gdf.set_crs(epsg=4326)
boundary_gdf = boundary_gdf.to_crs(land_use.crs)
boundary_gdf

,geometry
0,"POLYGON ((4556297.629 3290724.509, 4556290.005..."


In [7]:
# filter out land use outside of the boundary
land_use = gpd.overlay(land_use, boundary_gdf, how="intersection")

In [8]:
land_use = land_use.to_crs(epsg=4326)

In [9]:
land_use.to_file(landuse_file, driver="GeoJSON")

In [10]:
land_use = gpd.read_file(landuse_file)

In [11]:
legend = pd.read_csv(
    "/data/passive/research-data/general-data/u2018_clc2018_v2020_20u1/Legend//CLC_legend.csv",
    sep=";",
)

In [12]:
legend_map = legend.set_index("CLC_CODE")["LABEL3"].to_dict()

In [13]:
land_use["land_type"] = land_use["Code_18"].astype(int).map(legend_map)

In [14]:
land_use.land_type.unique()

array(['Discontinuous urban fabric', 'Industrial or commercial units',
       'Green urban areas', 'Sport and leisure facilities',
       'Non-irrigated arable land', 'Pastures',
       'Complex cultivation patterns', 'Broad-leaved forest',
       'Coniferous forest', 'Mixed forest', 'Natural grasslands',
       'Moors and heathland', 'Transitional woodland-shrub',
       'Water bodies', 'Continuous urban fabric',
       'Road and rail networks and associated land', 'Port areas',
       'Airports', 'Dump sites', 'Construction sites',
       'Land principally occupied by agriculture, with significant areas of natural vegetation',
       'Inland marshes', 'Water courses'], dtype=object)

In [15]:
residential = [
    "Continuous urban fabric",
    "Discontinuous urban fabric",
    # "Industrial or commercial units",
]

In [16]:
residential_land_use = land_use[land_use.land_type.isin(residential)]

In [17]:
# merge into one
residential_land_use_area = residential_land_use.geometry.unary_union

In [18]:
m = folium.Map(geo_meta.get_center_lat_lon())

folium.GeoJson(residential_land_use_area).add_to(m)

m

In [19]:
geo_meta.set_residential_area(residential_land_use_area)
geo_meta.save(geometa_path)